# Summarization

## Зависимости

In [1]:
# !conda update -n base -c conda-forge conda -y
# !conda install -c conda-forge ipywidgets -y
# !conda install -c conda-forge datasets -y
# !conda install -c conda-forge transformers -y
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y
# %pip install evaluate -qq
# %pip install rouge-score -qq
# %pip install nltk -qq
# %pip install absl-py -qq
# %pip install transformers[sentencepiece] -qq

## Подготовка данных

In [53]:
import random

import pandas as pd
from tqdm import tqdm

from datasets import load_dataset, ClassLabel, Dataset
from IPython.display import display, HTML

In [54]:
train = pd.read_csv('./dataset/train.csv')

In [55]:
def del_timestamps(text):
    text = text.split("]  ")[1:]
    return " ".join(text)

list_texts = []
for i in tqdm(range(500)):
    with open(f'./dataset/{i}.txt', 'r') as file:
        lines = file.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        list_texts.append(lines)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2801.42it/s]


In [56]:
list_texts[0][:100]

'Давай поспорим на выпуск, вот типа, что я точно угадаю цену одного из предметов. Ох, зима. Доктор Кл'

In [5]:
train['text'] = pd.Series(list_texts)
train = train.rename(columns={'description': 'summary'})

In [6]:
train.head()

,video_name,stt_name,category_name,title,summary,text
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,"Давай поспорим на выпуск, вот типа, что я точн..."
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,ты поедешь со мной в тверь это 2 выпуск В этом...
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...","Добрый вечер, дорогие друзья, с вами шоу «Хошл..."
3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...,СПОКОЙНАЯ МУЗЫКА Редактор субтитров Е.Воинова ...
4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...,ДИНАМИЧНАЯ МУЗЫКА ДИНАМИЧНАЯ МУЗЫКА АПЛОДИСМЕН...


In [72]:
video = pd.read_csv('./dataset/video.csv')
video = video.rename(columns={'description': 'summary', 'ru_videos': 'text'})
video = video.dropna()
video.shape

(119, 8)

In [73]:
video.head()

,Unnamed: 0,video_name,stt_name,category_name,title,summary,eng_videos,text
0,0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,A group of people are sitting at a table and o...,"Группа людей сидит за столом, и один из них ес..."
1,1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,A man is standing in front of a bunch of cardb...,"Мужчина стоит перед пучком картонных коробок, ..."
2,2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...",A man and a woman are sitting at a table and t...,Мужчина и женщина сидят за столом и говорят о ...
3,3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...,A person is driving a vehicle down a busy stre...,Человек управляет транспортным средством по ож...
4,4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...,A man and a woman are on a stage and a man is ...,"Мужчина и женщина находятся на сцене, и мужчин..."


In [74]:
dataset = Dataset.from_pandas(video)
dataset = dataset.train_test_split(test_size=0.2) 

/home/asr/miniconda3/envs/asr/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [75]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'video_name', 'stt_name', 'category_name', 'title', 'summary', 'eng_videos', 'text', '__index_level_0__'],
        num_rows: 95
    })
    test: Dataset({
        features: ['Unnamed: 0', 'video_name', 'stt_name', 'category_name', 'title', 'summary', 'eng_videos', 'text', '__index_level_0__'],
        num_rows: 24
    })
})

In [76]:
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [79]:
show_random_elements(dataset['train'])

,Unnamed: 0,video_name,stt_name,category_name,title,summary,eng_videos,text,__index_level_0__
0,61,61.mp4,61.txt,Искусство,"Артмеханика. Выпуск 8. Кастомайзинг как искусство + концерт Parks, Squares and Alleys","Описание выпуска: Бренд и художник, как им найти друг друга? Куда приведет тенденция к тотальной кастомизации? Скрытый фэшн от Жени Чеса и много еще чего в новом выпуске Артмеханики. Музыкальную часть программы украсили собой Parks, Squares and Alleys.","A man is giving a speech in front of a crowd and then a woman is standing next to him. A woman is talking to a group of people while a man is talking to a group of people. A group of people are sitting at a table and a woman is talking to them about something. A man and a woman are sitting at a table and a woman is talking to a man and a woman is talking to a man. A group of people are sitting at a table and playing a game of ping pong with each other. A man is standing in front of a camera and talking about a project he is working on. A man is sitting at a table and talking to a woman who is sitting next to him. A man is standing in front of a mirror and then a man is standing in front of a mirror and then a man is standing in front of a mirror A group of people are performing on stage while music is playing in the background and people are walking around. A man and a woman are playing rock, paper, scissors in front of a crowd of people. A man and a woman are playing a song on a stage in front of a crowd of people. A man is playing the drums on a stage in front of a crowd of people who are listening to him.","Мужчина выступает перед толпой, а затем женщина стоит рядом с ним. Женщина разговаривает с группой людей, в то время как мужчина разговаривает с группой людей. Группа людей сидит за столом, а женщина говорит с ними о чем-то. Мужчина и женщина сидят за столом, а женщина разговаривает с мужчиной, а женщина разговаривает с мужчиной. Группа людей сидит за столом и играет в пинг-понг друг с другом. Мужчина стоит перед камерой и говорит о проекте, над которым он работает. Мужчина сидит за столом и разговаривает с женщиной, которая сидит рядом с ним. Мужчина стоит перед зеркалом, а затем мужчина стоит перед зеркалом Группа людей выступает на сцене, в то время как музыка играет на заднем плане, и люди ходят вокруг. Мужчина и женщина играют рок, бумагу, ножницы перед толпой людей. Мужчина и женщина играют песню на сцене перед толпой людей. Мужчина и женщина играют на барабанах перед толпой людей.",61
1,81,81.mp4,81.txt,Блоги,Под другим углом | Выпуск 4 | Что такое звук?,"Бархатный голос, богатый тембр и, конечно, тёплый ламповый саунд! Почему звук такой разный? От чего зависит звучание и наше восприятие? Что мы вообще знаем об акустике, собственном голосе и как можем применить эти знания? От шёпота до УЗИ – Варя расскажет обо всём.",A woman is talking to a group of people and then a man is shown standing in front of a wall. A woman is standing in front of a computer screen and is talking to another woman in a foreign language. A man is standing in front of a stage and he is playing a song on a guitar.,Женщина стоит перед экраном компьютера и разговаривает с другой женщиной на иностранном языке. Мужчина стоит перед сценой и играет песню на гитаре.,81


## Обучение моделей

In [80]:
import gc
import os
import comet_ml

from accelerate import notebook_launcher
from accelerate.utils import set_seed

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import torch
import torch.optim as optim

In [81]:
os.environ["COMET_LOG_ASSETS"] = "True"

### facebook bart-large-cnn

In [82]:
model_checkpoint = "IlyaGusev/mbart_ru_sum_gazeta"

#### Tokenizer

In [83]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [84]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [85]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/95 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/24 [00:00<?, ? examples/s]

In [86]:
# block_size = tokenizer.model_max_length
BLOCK_SIZE = 1024
BATCH_SIZE = 2

In [87]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=BLOCK_SIZE, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=BLOCK_SIZE, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [88]:
lm_datasets = tokenized_datasets.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=8,
)

Map (num_proc=8):   0%|          | 0/95 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/24 [00:00<?, ? examples/s]

In [89]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [90]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

#### GPU runtime

In [91]:
torch.cuda.empty_cache()
gc.collect()

7

In [92]:
!nvidia-smi

Sat Sep 30 15:46:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:02:00.0 Off |                    0 |
| N/A   

#### Evaluate

In [93]:
import evaluate

rouge = evaluate.load("rouge")

In [94]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

#### Training

In [95]:
comet_ml.init( project_name = "KaiTube", experiment_name = "Summarization-rugazeta-gas-1024-video")

In [101]:
def training_function():
    
    training_args = Seq2SeqTrainingArguments(
        output_dir="IlyaGusev-mbart_ru_sum_gazeta-finetune-1024-gas-video",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=20,
        predict_with_generate=True,
        fp16=True,
        gradient_accumulation_steps=8,
        save_steps=55
    )
    
    set_seed(42)
    torch.manual_seed(7)
    
    optimizer = optim.AdamW(model.parameters(), training_args.learning_rate)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=training_args.learning_rate,
        steps_per_epoch=int(len(lm_datasets["train"])),
        epochs=training_args.per_device_train_batch_size,
        anneal_strategy='linear'
    )
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=lm_datasets["train"],
        eval_dataset=lm_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, scheduler)
    )
    trainer.train()
    return trainer

In [103]:
trainer = notebook_launcher(training_function, num_processes=2, mixed_precision='fp16')

Launching training on 2 GPUs.


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: You are trying to log string value as a metric. This is not recommended.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gumaonelove/kaitube/871884d2aedb4b04867d9f5c4228b4ed

You're using a MBartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a MBartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[W reducer.cpp:1300] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag res

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,4.651559,0.000000,0.000000,0.000000,0.000000,62.333300
2,No log,4.517689,0.000000,0.000000,0.000000,0.000000,64.416700
3,No log,4.457882,0.000000,0.000000,0.000000,0.000000,59.916700
4,No log,4.257483,0.000000,0.000000,0.000000,0.000000,82.333300
5,No log,4.124197,0.000000,0.000000,0.000000,0.000000,90.458300
6,No log,4.013084,0.000000,0.000000,0.000000,0.000000,89.375000
7,No log,3.936275,0.000000,0.000000,0.000000,0.000000,76.916700
8,No log,3.865460,0.000000,0.000000,0.000000,0.000000,60.291700
9,No log,3.799875,0.000000,0.000000,0.000000,0.000000,70.125000
10,No log,3.751960,0.000000,0.000000,0.000000,0.000000,73.333300


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,4.651559,0.000000,0.000000,0.000000,0.000000,62.333300
2,No log,4.517689,0.000000,0.000000,0.000000,0.000000,64.416700
3,No log,4.457882,0.000000,0.000000,0.000000,0.000000,59.916700
4,No log,4.257483,0.000000,0.000000,0.000000,0.000000,82.333300
5,No log,4.124197,0.000000,0.000000,0.000000,0.000000,90.458300
6,No log,4.013084,0.000000,0.000000,0.000000,0.000000,89.375000
7,No log,3.936275,0.000000,0.000000,0.000000,0.000000,76.916700
8,No log,3.865460,0.000000,0.000000,0.000000,0.000000,60.291700
9,No log,3.799875,0.000000,0.000000,0.000000,0.000000,70.125000
10,No log,3.751960,0.000000,0.000000,0.000000,0.000000,73.333300


RuntimeError: An issue was found when launching the training: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/asr/miniconda3/envs/asr/lib/python3.11/site-packages/torch/serialization.py", line 441, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/home/asr/miniconda3/envs/asr/lib/python3.11/site-packages/torch/serialization.py", line 668, in _save
    zip_file.write_record(name, storage.data_ptr(), num_bytes)
RuntimeError: [enforce fail at inline_container.cc:471] . PytorchStreamWriter failed writing file data/3: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/asr/miniconda3/envs/asr/lib/python3.11/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/asr/miniconda3/envs/asr/lib/python3.11/site-packages/accelerate/utils/launch.py", line 543, in __call__
    self.launcher(*arg

#### Inference

In [44]:
from transformers import pipeline

In [45]:
summarizer = pipeline("summarization", model="IlyaGusev-mbart_ru_sum_gazeta-finetune-1024/checkpoint-500/")

In [46]:
text = """[0.18s -> 2.70s]  Ты видел в жизни когда-нибудь Киркорова?
[2.70s -> 7.61s]  Да. Я работал на ЛАФ-радио, брат.
[7.61s -> 11.25s]  Вот это руками надо вытаскивать. Может, слайсами, Крис?
[11.25s -> 15.37s]  Я могу про себя много чего наговорить, но будет ли это правдой?
[15.37s -> 18.21s]  Ну, господи, как будто я другого спрашивал.
[18.21s -> 20.85s]  Я худший готовитель истории этого шоу.
[20.85s -> 23.09s]  Банданы, я считаю, что это говно.
[23.09s -> 24.69s]  До февраля? Сука!
[24.69s -> 27.69s]  Я думаю, с этого должен выпуск начинаться,
[27.69s -> 30.01s]  типа нарезок, как падает что-то.
[30.01s -> 31.01s]  Почему?
[34.57s -> 37.45s]  Привет всем пережившим и отдохнувшим...
[37.45s -> 38.45s]  Блин.
[38.45s -> 40.17s]  В гостях у нас сегодня...
[40.17s -> 41.65s]  Да, дурак? Да.
[41.65s -> 47.06s]  Привет всем пережившим и отдохнувшим.
[47.06s -> 48.78s]  Меня зовут Ярослав Грина,
[48.78s -> 51.82s]  это первый выпуск шоу Kitchen Stories в 2022 году.
[51.82s -> 53.82s]  Ну а в гостях у меня человек,
[53.82s -> 56.98s]  имя которого вы знаете, это Basic Boy, Макс Салют.
[56.98s -> 57.98s]  Салют.
[57.98s -> 60.06s]  Прежде чем мы начнем готовить,
[60.06s -> 62.86s]  я вкратце расскажу правила нашей программы.
[62.86s -> 65.10s]  Я жду от тебя классное настроение,
[65.14s -> 68.02s]  интересный рецепт и увлекательные истории.
[68.02s -> 69.02s]  Все будет.
[69.02s -> 70.70s]  Что мы сегодня готовим?
[70.70s -> 74.46s]  Сегодня мы готовим сельдь под шубой по рецепту моей мамы.
[74.46s -> 77.62s]  И картошечку со сливочным соусом, с грибочками.
[77.62s -> 80.22s]  По рецепту другой моей любимой женщины,
[80.22s -> 81.22s]  моя невеста Крис.
[81.22s -> 83.94s]  Сегодня будет два блюда, давайте кайфанем.
[83.94s -> 86.58s]  Мне кажется, у ребят уже сегодня потекли.
[86.58s -> 89.18s]  Расскажи, что нам для этого потребуется.
[89.18s -> 92.50s]  Селедочка, картошечка, свекла, грибочки, зеленечки,
[92.58s -> 94.06s]  луечек и майонезик.
[94.06s -> 97.50s]  И еще вот там чеснок, перец, соль, сливки и лучок.
[97.50s -> 98.50s]  Еще один.
[98.50s -> 99.50s]  С чего мы начнем?
[99.50s -> 103.22s]  Мы поставили остужаться картошечку, свеколку у нас варится.
[103.22s -> 105.62s]  Мы сейчас будем разделывать селедку.
[105.62s -> 109.58s]  Я не то чтобы мастер в этом деле, так что будем фристалить.
[109.58s -> 112.98s]  Я позвонил маме, автору этого рецепта, так скажем.
[112.98s -> 115.30s]  Она мне расписала, как это делать,
[115.30s -> 118.50s]  но я не очень уверен в своих силах в этом плане.
[118.50s -> 120.02s]  Надеюсь, все получится.
[120.02s -> 121.70s]  Для этого есть мои силы.
[122.38s -> 124.14s]  Ну что, народ, погнали.
[124.14s -> 127.74s]  А почему ты решил готовить именно селедку под шубой?
[127.74s -> 131.10s]  Во-первых, это такой новогодний рецепт, прикольный.
[131.10s -> 132.86s]  Мама постоянно его делала.
[132.86s -> 136.06s]  Это первое, что пришло в голову из таких рецептов,
[136.06s -> 137.62s]  которые я люблю реально.
[137.62s -> 141.18s]  Мама очень круто готовит, мне всегда очень нравилось.
[141.18s -> 144.78s]  И охота увековечить ее рецепты в пространстве интернета
[144.78s -> 146.06s]  и поделиться этим.
[146.06s -> 148.18s]  Я очень люблю все эти продукты.
[148.18s -> 150.30s]  Как часто ты сам готовишь?
[150.46s -> 153.90s]  Слушай, я не суперчасто готовлю, я люблю готовить.
[153.90s -> 157.30s]  Мне это нравится, но я все никак не могу заставить.
[157.30s -> 160.54s]  Ничего себе приготовить, кроме завтрака с утра.
[160.54s -> 163.14s]  Но у меня неплохо получается всячески.
[163.14s -> 166.46s]  Последнее, наверное, что я готовил, это панкейки.
[166.46s -> 168.90s]  Мне очень понравилось, если честно.
[168.90s -> 172.62s]  А так я все никак не могу заставить, просто это делать.
[172.62s -> 177.18s]  Мне реально нравится этот процесс, очень умиротворяет медитативно.
[177.22s -> 179.42s]  Здесь проходимся, тоже глубоко.
[179.42s -> 181.02s]  Насколько я понял, да.
[181.02s -> 183.26s]  Чтобы у нас две части получилось,
[183.26s -> 186.18s]  и там где-то посредине останется позвоночник.
"""

In [47]:
import re
reg = re.compile('[^а-яА-Я ]')
text = reg.sub('', text)

In [48]:
summarizer(text[:1000])

[{'summary_text': 'Добро пожаловать на шоу «Макс Салют»! Шоу, в котором обсуждаются актуальные новости страны и мира, в котором обсуждаются актуальные новости страны и мира. В этом выпуске мы пригласим в нашу студию одного из самых ярких и харизматичных музыкантов России, а также приготовим традиционное блюдо из сельдьё под шубой по рецепту мамы. Приятного аппетита!'}]

#### Inference

In [36]:
summarizer = pipeline("summarization", model="IlyaGusev/mbart_ru_sum_gazeta")

In [37]:
summarizer(text)

[{'summary_text': 'Стеклянная ваза с нарисованными на ней цветами Стеклянная ваза с нарисованным на ней лицом Стеклянная скульптура с надписью Я не могу видеть изображение выше Чернобелое фото человека сидящего на полу Стеклянная скульптура слона сидящего на столе Стеклянная скульптура слона сидящего на столе Стеклянная ваза с нарисованным на ней лицом Стеклянная ваза стоит на столе Синебелый уличный фонарь и дерево Крупный план часов на стене здания Крупный план цветка на фоне неба Чернобелая фотография человека стоящего на коленях на земле Крупный план уличного знака на фоне голубого неба Крупный план знака остановки на фоне неба Уличная сцена с изображением знака Стоп Чернобелое фото трех человек держащих плюшевых мишек Группа людей стоящих рядом друг с'}]